# Convert Bin Files to PNGs

Frames are save in 200-frame .bin files to increase speed during collection. Before analyzing, its faster to save a whole set of .bin files to images.  

Set this to run and leave it. It takes a LONG time.

## Fix Permissions
When collecting, jupyter has to run as SUDO for keyboard communication. This makes some files and folders with sudo permissions, even for read. 

In Terminal:
'sudo chmod -R 777 /data_repo/bravo'

## Dependencies

In [1]:
import os, stat
import numpy as np
import matplotlib.pyplot as plt
import imp

import utils.run_analysis as ana

## Parameters

*** NOTE: REALLY NEED TO CHECK ON THE VALUES FOR HORIZONTAL AND VERTICAL FOV ***

In [2]:
base_dir = '/data_repo/bravo'
data_dir = os.path.join(base_dir, 'raw_data')
output_dir = os.path.join(base_dir, 'analysis')
camera = 'buddy'

fps = 200

img_dims=(1544,2064)
save_batchsize = 200

## Specify Directories

In [3]:
subject = 'buddy'
trial = 'cellphone_1'
camera = 'cy'
trial_directory = os.path.join(data_dir, subject, trial, 'pre')
camera_dir = os.path.join(trial_directory,'scene_camera')

analysis_folder = os.path.join(output_dir, subject, trial,'')

#three trace conditions: true, none, temporal_match, spatial_match 
trace='none'

In [4]:
import cv2
def convert_bin_pngs(filename, first_fnum, save_batchsize, save_folder,  dims=(1544,2064)):
    '''
    Take a file saved in .bin format from a ximea camera, and convert it to png images.
    Parameters:
        filename (str): file to be converted
        save_folder (str): folder to save png files
        im_shape (2pule ints): shape of image
        img_format (str): Image format files are saved
    Returns:
        None
    '''
    nbytes = np.prod(dims)
    
    with open(filename, 'rb') as fn:
        bs = fn.read(1)
        for i in range(first_fnum, first_fnum+save_batchsize):
            save_filepath = os.path.join(save_folder, f'frame_{i}.png')
            binary_img = []
            for b in range(nbytes):
                binary_img.append(int.from_bytes(fn.read(1),'big'))
            binary_img = np.array(binary_img)
            cimage = cv2.flip(cv2.cvtColor(np.uint8(binary_img.reshape(dims)),cv2.COLOR_BayerGR2BGR),-1)
            cv2.imwrite(save_filepath, cimage)


In [5]:
def convert_trial_directory(camera_dir, camera, save_batchsize, analysis_folder):
    frame_start = 0
    bin_file = os.path.join(camera_dir,camera,f'frames_{frame_start}_{frame_start+save_batchsize-1}.bin')
    cam_folder = os.path.join(analysis_folder,camera,'pngs')
    try:
        os.makedirs(cam_folder)
    except:
        print('already made cam folder!')

    print(f'Converting bin to png for folder {os.path.join(camera_dir,camera)}')
    print(f'Each * is {save_batchsize} frames...')
    while(os.path.isfile(bin_file)):
        print('*')
        convert_bin_pngs(bin_file, frame_start, save_batchsize, cam_folder, dims=(1544,2064))
        frame_start += save_batchsize
        bin_file = os.path.join(camera_dir,camera,f'frames_{frame_start}_{frame_start+save_batchsize-1}.bin')
    print('Done!')

## Run on One Trial

In [6]:
convert_trial_directory(camera_dir, camera, save_batchsize, analysis_folder)

Converting bin to png for folder /data_repo/bravo/raw_data/buddy/cellphone_1/pre/scene_camera/cy
Each * is 200 frames...
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
Done!


## Run Script on All Trials

In [9]:
import itertools as it
subjects = ['buddy']
tasks = ['chat','book','cellphone','laptop']
repititions = ['1']
iterator = it.product(subjects, tasks, repititions)

for iteration in iterator:
    subject, task, rep = iteration
    trial = f'{task}_{rep}'
    if(os.path.isdir(os.path.join(base_dir,'raw_data', subject, trial))):
        camera_dir = os.path.join(base_dir,'raw_data', subject, trial,'pre','scene_camera')
        analysis_folder = os.path.join(output_dir, subject, trial)
        convert_trial_directory(camera_dir, camera, save_batchsize, analysis_folder)

Converting bin to png for folder /data_repo/bravo/raw_data/buddy/chat_1/pre/scene_camera/cy
Each * is 200 frames...
*


KeyboardInterrupt: 